In [ ]:
import os
import glob
import collections
import pickle
from pathlib import Path

from astropy.io import ascii
from vega import VegaInterface
import emcee
import corner
import yaml
import scipy.optimize, scipy.stats
%pylab inline

import constants
from MCMC_Run import one_dimensional_log_lik

In [ ]:
LABEL_MAPPING = {
    'bias_QSO': r'$b_{gal}$', 
    'beta_QSO': r'$\beta_{gal}$', 
    'sigma_velo_disp_gauss_QSO': r'$\sigma_z$',
    'drp_QSO': r'$\delta_z$',
    'bias_hcd': r'$b_{DLA}$',
    'beta_hcd': r'$\beta_{DLA}$',
    'L0_hcd': r'$L_{DLA}$'
}

SURVEY_TITLE_MAPPING = {
    '3dhst': '3D-HST',
    'clamato': 'CLAMATO',
    'mosdef': 'MOSDEF',
    'vuds': 'VUDS',
    'zDeep': 'COSMOS-zDEEP'
}

STACKED_MASS_BIN_TITLE_MAPPING = {
    'lowmass': 'LM',
    'medmass': 'MM',
    'highmass': 'HM'
}

constants.MCMC_DIR_BASE = '/global/homes/b/bzh/clamato-xcorr/data/split-mcmc'

In [ ]:
def plot_model_w_data(survey_name, param_dict, model_num_params,
                      SigMin = 0., SigMax = 30.,
                      PiMin = -30., PiMax = 30.,
                      one_dimensional=False):

    PiBins0 = ascii.read(Path(constants.XCORR_DIR_BASE) / 'bins23_pi_0-30hMpc.txt')
    SigBins0 = ascii.read(Path(constants.XCORR_DIR_BASE) / 'bins10_sigma_0-30hMpc.txt')
    cosmo = constants.COSMOLOGY
    
    XCorr_obs = np.load(Path(constants.XCORR_DIR_BASE) / 'split' / f'xcorr_{survey_name}_globalf_{constants.DATA_VERSION}.npy')

    PiEdges = PiBins0['pi_edges'].data
    SigEdges = SigBins0['sigma_edges'].data

    SigEdgesVec, PiEdgesVec = np.meshgrid(SigEdges, PiEdges)

    SigEdgesPlot = SigEdgesVec / (np.ones(np.shape(SigEdgesVec))*[cosmo.h])
    PiEdgesPlot = PiEdgesVec / (np.ones(np.shape(PiEdgesVec))*[cosmo.h])
    
    SigCentersPlot = (SigEdges[1:] + SigEdges[:-1]) / 2
    SigCentersPlot /= cosmo.h

    vega = VegaInterface(Path(constants.MCMC_DIR_BASE) / survey_name.split('_')[0] / f'main_{survey_name}.ini')
    if one_dimensional:
        VegaInterface.log_lik = one_dimensional_log_lik
    
    for k, v in param_dict.items():
        vega.params[k] = v
    if 'beta_QSO' not in param_dict and 'bias_eta_QSO' not in param_dict:
        vega.params['beta_QSO'] = vega.params['growth_rate'] / vega.params['bias_QSO']
    XModel = vega.compute_model()['qsoxlya'].reshape(*XCorr_obs.shape)
    XModelPlot = XModel.T
        
    # Verify that the model is now in the same shape as the observed (which should be in the 
    # 'original' shape)
    XCorrPlot = np.transpose(XCorr_obs)
    if not np.array_equal(np.shape(XModelPlot),np.shape(XCorrPlot)):
        if np.array_equal(np.shape(np.transpose(XCorrPlot)),np.shape(XModelPlot)):
            XCorrPlot = np.transpose(XCorrPlot)
        else:
            print('Input XCorr array not compatible with model array!')
            
    if one_dimensional:
        _, (chi2, reduced_covar, mask_nonflat) = vega.log_lik(return_aux=True)

        # Reduce xcorrs.
        XCorrPlot = np.ma.array(XCorrPlot, mask=mask_nonflat.T).sum(axis=0)
        XModelPlot = np.ma.array(XModelPlot, mask=mask_nonflat.T).sum(axis=0)
        assert len(XCorrPlot) == len(SigCentersPlot)
        assert XCorrPlot.ndim == 1

        plt.errorbar(SigCentersPlot, XCorrPlot, yerr=np.sqrt(np.diag(reduced_covar)), color='black', label='Observed')
        plt.plot(SigCentersPlot, XModelPlot, color='red', label='Best-fit model')
        plt.xlabel(r'$\sigma\; (\mathrm{cMpc})$')
        plt.ylabel(r'$\pi$-summed cross-correlation')
        plt.title(f'Reduced $\chi^2$ = {chi2 / (np.sum(~XCorrPlot.mask) - model_num_params):.3f}', y=0.9, fontsize=10)
    else:
        fig, (ax1, ax2) = plt.subplots(1,2,figsize=(5,5))

        ax1.pcolormesh(SigEdgesPlot, PiEdgesPlot, XCorrPlot,cmap='jet_r',vmin=-0.2, vmax=0.1 )
        ax1.set_aspect('equal')
        ax1.set_xlim(0., SigMax)
        ax1.set_ylim(PiMin, PiMax)
        ax1.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
        ax1.set_ylabel(r'$\pi\; (\mathrm{cMpc})$')
        # ax1.set_title(survey_name,fontsize=10)

        ax2.pcolormesh(SigEdgesPlot, PiEdgesPlot, XModelPlot,cmap='jet_r',vmin=-0.2, vmax=0.1 )
        ax2.set_aspect('equal')
        ax2.set_xlim(SigMin, SigMax)
        ax2.set_ylim(PiMin, PiMax)
        ax2.set_xlabel(r'$\sigma\; (\mathrm{cMpc})$')
        # ax2.set_title(f'chi-sq = {vega.chi2():.1f}',fontsize=10)
        # print(vega.corr_items['qsoxlya'].rp_rt_grid[1][~vega.data['qsoxlya'].mask])
        fig.suptitle(f'Reduced $\chi^2$ = {vega.chi2() / (np.sum(vega.data["qsoxlya"].mask) - model_num_params):.3f}', y=0.9, fontsize=10)

In [ ]:
def corner_plot(survey_name, chain_suffix='', config_suffix=None, show_grid_fit=False, one_dimensional=False, **kwargs):
    backend = emcee.backends.HDFBackend(os.path.join(constants.MCMC_DIR_BASE, survey_name, f'chain_{survey_name}{chain_suffix}.hdf5'), read_only=True)
    config = None
    if config_suffix is None:
        config_suffix = chain_suffix
    for p in glob.glob('mcmc_cfg/*.yaml'):
        if p.split('/')[-1].casefold() == f'{survey_name.casefold()}_split{config_suffix}.yaml':
            with open(p, 'r') as f:
                config = yaml.safe_load(f)
    if not config:
        raise RuntimeError
    if 'beta_QSO' in config['initial'] and config['initial']['beta_QSO'] is None:
        del config['initial']['beta_QSO']
    tau = backend.get_autocorr_time(quiet=True)
    burnin = int(2 * np.max(tau))
    thin = int(0.5 * np.min(tau))
    # thin = int(np.max(tau))
    chain_len = len(backend.get_chain())
    samples = backend.get_chain(discard=burnin, flat=True, thin=thin)
    
    print(f"Autocorrelation time: {tau}")
    print(f"Chain length: {chain_len}; Chain length / 50: {chain_len / 50:.2f}")
    print("Burn-in: {0}".format(burnin))
    print("Thin: {0}".format(thin))
    print("Flat chain shape: {0}; {1:.1f} autocorrelation times.".format(samples.shape, samples.shape[0] / np.max(tau)))
    
    final_medians = np.median(samples, axis=0)
    unflat_samples = backend.get_chain(discard=burnin, flat=False, thin=thin)
    # print(unflat_samples.shape)
    for i in range(1, unflat_samples.shape[0]):
        subsample = unflat_samples[:i].reshape(-1, samples.shape[-1])
        if np.allclose(np.median(subsample, axis=0), final_medians, rtol=0.05):
            print(f'# chain iterations to arrive at ~5% parameter estimate: {i * thin + burnin}')
            break
    
    if show_grid_fit:
        truths = config['initial'].values()
    else:
        truths = None
        
    # For corner plot, multiply delta_z by -1 so we're consistent with definitions.
    delta_z_inverted_samples = np.copy(samples)
    try:
        assert 'drp_QSO' in config['initial']
        delta_z_ind = -1
        delta_z_inverted_samples[:, delta_z_ind] *= -1
        if truths:
            truths[delta_z_ind] *= -1
    except AssertionError:
        pass
    
    # Divide dispersion by 2 to get to what we consider z-dispersion.
    disp_corrected_samples = np.copy(delta_z_inverted_samples)
    try:
        assert 'sigma_velo_disp_gauss_QSO' in config['initial']
        sigma_z_ind = -2
        disp_corrected_samples[:, sigma_z_ind] /= 2
        if truths:
            truths[sigma_z_ind] /= 2
    except AssertionError:
        pass
    
    corner_labels = []
    for mass_bin in constants.STACKED_BIN_TITLES:
        corner_labels.append(f"{STACKED_MASS_BIN_TITLE_MAPPING[mass_bin]} {LABEL_MAPPING['bias_QSO']}")
    if 'sigma_velo_disp_gauss_QSO' in config['initial']:
        corner_labels.append(LABEL_MAPPING['sigma_velo_disp_gauss_QSO'])
    if 'drp_QSO' in config['initial']:
        corner_labels.append(LABEL_MAPPING['drp_QSO'])
    
    corner.corner(disp_corrected_samples, labels=corner_labels, show_titles=True, 
                  truths=truths, truth_color='red',
                  **kwargs)

    if one_dimensional:
        bias_last_ind = None
    else:
        bias_last_ind = -2
    bias_corrcoef = np.corrcoef(samples[:,  :bias_last_ind], rowvar=False)
    plt.text(0.5, 0.8, f'Bias correlation matrix:\n{bias_corrcoef}', transform=plt.gcf().transFigure)
    
    plt.suptitle(SURVEY_TITLE_MAPPING[survey_name], x=0.54, weight='bold', fontsize=14)
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', f'titledcorner_{survey_name}{chain_suffix}.png'))
    plt.suptitle('')
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', f'corner_{survey_name}{chain_suffix}.pdf'))
    plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', f'corner_{survey_name}{chain_suffix}.png'))
    plt.show()
    for i, mass_bin in enumerate(constants.STACKED_BIN_TITLES):
        median_params = {
            'bias_QSO': np.median(samples[:, i])
        }
        if 'sigma_velo_disp_gauss_QSO' in config['initial']:
            median_params['sigma_velo_disp_gauss_QSO'] = np.median(samples[:, sigma_z_ind])
        if 'drp_QSO' in config['initial']:
            median_params['drp_QSO'] = np.median(samples[:, delta_z_ind])
            
        if 'fixed' in config and config['fixed']:
            for k, p in config['fixed'].items():
                median_params[k] = p
        plot_model_w_data(f'{survey_name}_{mass_bin}', median_params, samples.shape[1] - (len(constants.STACKED_BIN_TITLES) - 1), one_dimensional=one_dimensional)
        plt.tight_layout()
    
        plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', f'bestmodel_{survey_name}_{mass_bin}{chain_suffix}.pdf'))
        plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', f'bestmodel_{survey_name}_{mass_bin}{chain_suffix}.png'))
        if one_dimensional:
            plt.title(f'{STACKED_MASS_BIN_TITLE_MAPPING[mass_bin]} {SURVEY_TITLE_MAPPING[survey_name]} | ' + plt.gca().get_title(), y=0.9, fontsize=10)
        else:
            plt.suptitle(f'{STACKED_MASS_BIN_TITLE_MAPPING[mass_bin]} {SURVEY_TITLE_MAPPING[survey_name]} | ' + plt.gcf()._suptitle.get_text(), y=0.9, fontsize=10)
        plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', f'titledbestmodel_{survey_name}_{mass_bin}{chain_suffix}.png'))
        plt.show()
    return samples

In [ ]:
sample_2d = {}
sample_1d = {}

In [ ]:
sample_2d['clamato'] = corner_plot('clamato', chain_suffix='', config_suffix='')

In [ ]:
sample_1d['clamato'] = corner_plot('clamato', chain_suffix='_oned', one_dimensional=True)

In [ ]:
sample_2d['vuds'] = corner_plot('vuds')

In [ ]:
sample_1d['vuds'] = corner_plot('vuds', chain_suffix='_oned', one_dimensional=True)

In [ ]:
sample_2d['zDeep'] = corner_plot('zDeep')

In [ ]:
sample_1d['zDeep'] = corner_plot('zDeep', chain_suffix='_oned', one_dimensional=True)

In [ ]:
sample_2d['mosdef'] = corner_plot('mosdef')

In [ ]:
sample_1d['mosdef'] = corner_plot('mosdef', chain_suffix='_oned', one_dimensional=True)

In [ ]:
plt.figure(figsize=(1.5, 5))
img = plt.imshow(np.array([[-0.2, 0.1]]), cmap='jet_r')
img.set_visible(False)
plt.axis('off')
plt.colorbar(fraction=1)
plt.savefig(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc', 'bestmodel_colorbar.png'))

In [ ]:
pwd = os.getcwd()
os.chdir(os.path.join(constants.FIG_DIR_BASE, 'split-mcmc'))
os.system('./create-mosaic.sh')
os.chdir(pwd)

# Marginalized bias distributions

In [ ]:
def hist_mode(data, n_bins):
    '''Estimates mode of data from a histogram

    Args:
        data: Samples.
        n_bins: Number of histogram bins to use.

    Returns:
        Mode estimated from histogram.
    '''
    hist, edges = np.histogram(data, n_bins)
    # Center the bin edges.
    edges += (edges[1] - edges[0]) / 2
    return edges[np.argmax(hist)]

def variance_from_mode(data, n_bins):
    '''Compute variance, but using histogram-estimated mode as central value,
    instead of mean.

    Args:
        data: Samples.
        n_bins: Number of histogram bins to use for estimating the mode.

    Returns:
        "Variance" using mode as central value.
    '''
    mode = hist_mode(data, n_bins)
    return np.sum((data - mode)**2) / (len(data) - 1)

def fit_truncated_normal(data):
    '''Fit a truncated normal distribution to data using maximum-likelihood estimation,
    via LFBGS-B.

    Args:
        data: Samples.

    Returns:
        fit_params: Best-fit parameters of truncated normal model; mean and standard deviation.
        Boolean on whether optimizer converged or not.
    '''
    if data.ndim > 1:
        data = data.flatten()
    mode = hist_mode(data, 20)
    def target_log_prob_fn(theta):
        loc, scale = theta
        # Support is in terms of z-values for whatever reason (thanks, scipy).
        # Fix truncation to [0, inf).
        a, b = (0 - loc) / scale, np.inf
        return -np.sum(scipy.stats.truncnorm.logpdf(data, a, b, loc=loc, scale=scale))
    # Optimizer fails spectacularly sometimes (fits 0 mean, extremely large sigma), so try to avoid this by bounding.
    mean_bounds = (0, np.max(data))
    sigma_bounds = (np.std(data), np.max(data) - np.min(data))
    result = scipy.optimize.minimize(target_log_prob_fn, 
                                     (mode, variance_from_mode(data, 20)**0.5),
                                     bounds=((mean_bounds[0], mean_bounds[1]), (sigma_bounds[0], sigma_bounds[1])))
    return result.x

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sample_2d_fits = {k: [] for k in sample_2d.keys()}

def plot_normal(axis, x, loc, scale, color):
    a, b = (0 - loc) / scale, (np.inf - loc) / scale
    axis.plot(x, scipy.stats.truncnorm.pdf(x, a, b, loc=loc, scale=scale), color=color, alpha=1)
    
def get_1sigma_bounds(loc, scale, percentile=(16, 50, 84)):
    a, b = (0 - loc) / scale, (np.inf - loc) / scale
    truncnorm = scipy.stats.truncnorm(a, b, loc=loc, scale=scale)
    return truncnorm.ppf(np.array(percentile) / 100)

for survey, samples in sample_2d.items():
    for i, label in enumerate(['Low mass', 'Medium mass', 'High mass']):
        _, _, patches = axes[i].hist(samples[:, i], bins=100, alpha=0.4, label=survey, density=True)
        axes[i].set_xlim(0, None)
        patchcolor = patches.patches[0].get_facecolor()
        mean, sigma = fit_truncated_normal(samples[:, i])
        plot_normal(axes[i], np.linspace(*axes[i].get_xlim(), 1000), mean, sigma, patchcolor)
        axes[i].set_title(label)
        sample_2d_fits[survey].append((mean, sigma))
plt.suptitle('2D cross-correlation')
plt.legend()
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sample_1d_fits = {k: [] for k in sample_2d.keys()}

for survey, samples in sample_1d.items():
    for i, label in enumerate(['Low mass', 'Medium mass', 'High mass']):
        _, _, patches = axes[i].hist(samples[:, i], bins=100, alpha=0.4, label=survey, density=True)
        axes[i].set_xlim(0, None)
        patchcolor = patches.patches[0].get_facecolor()
        mean, sigma = fit_truncated_normal(samples[:, i])
        plot_normal(axes[i], np.linspace(*axes[i].get_xlim(), 1000), mean, sigma, patchcolor)
        axes[i].set_title(label)
        sample_1d_fits[survey].append((mean, sigma))
plt.suptitle('1D cross-correlation')
plt.legend()
plt.tight_layout()

## Combine PDFs (naive multiplication)

In [ ]:
global_hist_range = (0, 10)
global_hist_nbins = 100
global_hist_width = (global_hist_range[1] - global_hist_range[0]) / global_hist_nbins

global_hist_bin_edges = np.histogram([], bins=global_hist_nbins, range=global_hist_range)[1]

def compute_multiplied_histogram(samples_list, ind):
    # Has shape (# surveys, # histogram bins).
    hist_list = []
    for samples in samples_list:
        param = samples[:, ind]
        hist_list.append(np.histogram(param, bins=global_hist_nbins, range=global_hist_range, density=True)[0])
    # Reduce by multiplying along the first (survey) axis.
    combined_hist = np.prod(np.array(hist_list), axis=0)
    assert len(combined_hist) == global_hist_nbins
    # Renormalize the histogram.
    combined_hist /= np.sum(combined_hist * global_hist_width)
    assert np.isclose(np.sum(combined_hist) * global_hist_width, 1)
    return combined_hist

In [ ]:
# https://math.stackexchange.com/questions/114420/calculate-the-product-of-two-gaussian-pdfs
def compute_multiplied_gaussian_pdfs(theta_1, theta_2):
    mu_1, sig_1 = theta_1
    mu_2, sig_2 = theta_2
    sig = (sig_1**-2 + sig_2**-2)**-0.5
    mu = (mu_1 * sig_2**2 + mu_2 * sig_1**2) / (sig_1**2 + sig_2**2)
    return mu, sig

# # https://stackoverflow.com/questions/5558418/list-of-dicts-to-from-dict-of-lists
def dictoflist_to_listofdict(dol):
    return [dict(zip(dol,t)) for t in zip(*dol.values())]

combined_norm_params_2d = []

for param_survey_dict in dictoflist_to_listofdict(sample_2d_fits):
    param_list = param_survey_dict.values()
    combined_theta = None
    for p in param_list:
        if combined_theta is None:
            combined_theta = p
        else:
            combined_theta = compute_multiplied_gaussian_pdfs(combined_theta, p)
    combined_norm_params_2d.append(combined_theta)
    
combined_norm_params_1d = []

for param_survey_dict in dictoflist_to_listofdict(sample_1d_fits):
    param_list = param_survey_dict.values()
    combined_theta = None
    for p in param_list:
        if combined_theta is None:
            combined_theta = p
        else:
            combined_theta = compute_multiplied_gaussian_pdfs(combined_theta, p)
    combined_norm_params_1d.append(combined_theta)

In [ ]:
print(combined_norm_params_2d)

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=True, figsize=(5, 8))

bias_onesig_percentiles_2d_hist = []
bias_onesig_percentiles_1d_hist = []

bias_onesig_percentiles_2d = []
bias_onesig_percentiles_1d = []

def binned_percentile(h, percentile):
    cs = np.cumsum(h)
    bin_idx = np.argwhere(cs >= percentile / 100 * np.sum(h))[0][0]
    return global_hist_bin_edges[bin_idx] + (global_hist_width / 2)

for i, (label, color) in enumerate(zip(['Low mass', 'Medium mass', 'High mass'], ['green', 'orange', 'red'])):
    plt.sca(axes[i])
    plt.xlim(*global_hist_range)

    mult_hist_2d = compute_multiplied_histogram(sample_2d.values(), i)
    mult_hist_1d = compute_multiplied_histogram(sample_1d.values(), i)
    
    patch = plt.stairs(mult_hist_2d, global_hist_bin_edges, label=f'2D {label}', fill=True, color=color, alpha=0.5)
    patchcolor = patch.get_facecolor()
    plot_normal(plt.gca(), np.linspace(*plt.gca().get_xlim(), 1000), *combined_norm_params_2d[i], patchcolor)
    
    plt.stairs(mult_hist_1d, global_hist_bin_edges, label=f'1D {label}',
               color='black', hatch='/')
    plot_normal(plt.gca(), np.linspace(*plt.gca().get_xlim(), 1000), *combined_norm_params_1d[i], 'black')
    
    plt.legend()
    plt.ylabel('PDF')
    
    bias_onesig_percentiles_2d_hist.append((binned_percentile(mult_hist_2d, 16), binned_percentile(mult_hist_2d, 50), binned_percentile(mult_hist_2d, 84)))
    bias_onesig_percentiles_1d_hist.append((binned_percentile(mult_hist_1d, 16), binned_percentile(mult_hist_1d, 50), binned_percentile(mult_hist_1d, 84)))
    # Use multiplied continuous RVs instead.
    bias_onesig_percentiles_2d.append(get_1sigma_bounds(*combined_norm_params_2d[i]))
    bias_onesig_percentiles_1d.append(get_1sigma_bounds(*combined_norm_params_1d[i]))
    
plt.xlabel(r'$b_{gal}$')
plt.tight_layout()

In [ ]:
print(bias_onesig_percentiles_2d_hist)
print(bias_onesig_percentiles_1d_hist)

print('----------')

print(bias_onesig_percentiles_2d)
print(bias_onesig_percentiles_1d)

## Overplot really rough halo mass-stellar mass estimates onto LATIS figure

In [ ]:
with open(os.path.join(constants.BIAS_DIR_BASE, 'theoretical', 'sim_bias_binwidth_0.1_binstep_0.1.pickle'), 'rb') as f:
    theoretical_mass, theoretical_bias = pickle.load(f)
    
hmass_bias_theor_interp_func = lambda b: np.interp(b, theoretical_bias, theoretical_mass)

hmass_onesig_percentiles_theor_2d = hmass_bias_theor_interp_func(np.array(bias_onesig_percentiles_2d))
hmass_onesig_percentiles_theor_1d = hmass_bias_theor_interp_func(np.array(bias_onesig_percentiles_1d))

In [ ]:
with open(os.path.join(constants.BIAS_DIR_BASE, 'vega', f'hmass_bias_CLAMATO.pickle'), 'rb') as f:
    vega_mass, vega_bias = pickle.load(f)
    
hmass_bias_vega_interp_func = lambda b: np.interp(b, vega_bias, vega_mass)

hmass_onesig_percentiles_vega_2d = hmass_bias_vega_interp_func(np.array(bias_onesig_percentiles_2d))
hmass_onesig_percentiles_vega_1d = hmass_bias_vega_interp_func(np.array(bias_onesig_percentiles_1d))

In [ ]:
# Taken from stacked_observations.ipynb
bin_smass = [9.013238636363637, 9.7173801010101, 10.30939241813602]
yerr_theor_2d = np.abs(hmass_onesig_percentiles_theor_2d[:, (0, 2)] - hmass_onesig_percentiles_theor_2d[:, 1, np.newaxis])
yerr_theor_1d = np.abs(hmass_onesig_percentiles_theor_1d[:, (0, 2)] - hmass_onesig_percentiles_theor_1d[:, 1, np.newaxis])

yerr_vega_2d = np.abs(hmass_onesig_percentiles_vega_2d[:, (0, 2)] - hmass_onesig_percentiles_vega_2d[:, 1, np.newaxis])
yerr_vega_1d = np.abs(hmass_onesig_percentiles_vega_1d[:, (0, 2)] - hmass_onesig_percentiles_vega_1d[:, 1, np.newaxis])

plt.errorbar(bin_smass, hmass_onesig_percentiles_theor_1d[:, 1], yerr=yerr_theor_1d.T, 
             lw=0, elinewidth=2, capsize=8, markeredgewidth=2, label='1D (theoretical bias curve)', color='orange', fmt='o')
plt.errorbar(bin_smass, hmass_onesig_percentiles_theor_2d[:, 1], yerr=yerr_theor_2d.T, 
             lw=0, elinewidth=2, capsize=8, markeredgewidth=2, label='2D (theoretical bias curve)', color='red', fmt='o')
plt.xlim(8.5, 10.7)
plt.ylim(9, 13)
plt.xticks([])
plt.yticks([])
plt.legend(loc='lower right')
plt.savefig('/global/homes/b/bzh/clamato-xcorr/figures/split-mcmc/hmass-smass-theoretical.png', transparent=True, dpi=400)

In [ ]:
plt.errorbar(bin_smass, hmass_onesig_percentiles_vega_1d[:, 1], yerr=yerr_vega_1d.T, 
             lw=0, elinewidth=2, capsize=8, markeredgewidth=2, label='1D (Vega-fitted bias curve)', color='purple', fmt='o')
plt.errorbar(bin_smass, hmass_onesig_percentiles_vega_2d[:, 1], yerr=yerr_vega_2d.T, 
             lw=0, elinewidth=2, capsize=8, markeredgewidth=2, label='2D (Vega-fitted bias curve)', color='pink', fmt='o')

plt.xlim(8.5, 10.7)
plt.ylim(9, 13)
plt.xticks([])
plt.yticks([])
plt.legend(loc='lower right')
plt.savefig('/global/homes/b/bzh/clamato-xcorr/figures/split-mcmc/hmass-smass-vegacurve.png', transparent=True, dpi=400)